In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

%pylab inline


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 3.6.4 (default, Jan 28 2018 00:00:00)
SparkSession available as 'spark'.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.functions import desc
from pyspark.ml.feature import Tokenizer, RegexTokenizer, HashingTF, IDF
from pyspark.sql.types import IntegerType
import json

In [3]:
df = spark.read.format("json").load("/labs/lab07data/DO_record_per_line.json")

In [ ]:
df = spark.read.format("json").load("/labs/lab07data/DO_record_per_line.json")
regex = re.compile(r'[\w\d]{2,}', re.U)

f1 = udf(lambda x: ' '.join(regex.findall(x.lower())))

ndf = df.withColumn("ndesc", f1(df["desc"])).select('id', 'ndesc','lang')

tokenizer = Tokenizer(inputCol="ndesc", outputCol="words")
tokenized = tokenizer.transform(ndf)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(tokenized)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

#rescaledData.show(10)

def cos_sim_sp(a,b):
    if a.norm(2) != 0 and  b.norm(2) != 0:
        res = a.dot(b)/(a.norm(2)*b.norm(2))
    else:
        res = 0
    return float(res)

fun_cos = udf(lambda x: cos_sim_sp(x, course_1[0]["features"]), FloatType())

ids_to_get = [15516,22777,13131,5660,965,877]
data_to_save = {}
for i in ids_to_get:
    course_1 = rescaledData.filter("id = {}".format(i)).select("features").collect()
    fun_cos = udf(lambda x: cos_sim_sp(x, course_1[0]["features"]), FloatType())
    if i == 877 or i == 965:
        data = rescaledData.withColumn("course_1", fun_cos(rescaledData['features'])).filter("id != {}".format(i)).filter("lang='ru'").sort(desc("course_1")).limit(10)
    else:
        data = rescaledData.withColumn("course_1", fun_cos(rescaledData['features'])).filter("id != {}".format(i)).sort(desc("course_1")).limit(10)
    my_ids = []
    for i_like in data.select("id").collect():
        my_ids.append(i_like["id"])
    data_to_save[str(i)]=my_ids



In [ ]:
import json
with open('lab07.json', 'w') as fp:
    json.dump(data_to_save, fp)